# The Cross-Entroy Method

1. Despite the fact that it is much less famous than other tools in the RL practitioner's toolbox, such as `deep Q-network (DQN)` or `advantage actor-critic (A2C)`, the `cross-entropy (CE) method` has its own strength:
    1. easy, simple to follow. Pytorch implementation with just $<100$ lines of code
    2. has good convergence, works well in simple environments that 
        - don't require us to learn complex. multistep policies, and
        - have short episodes with frequent rewards

2. We'll cover:
    1. the practical side of the `cross-entropy (CE) method`
    2. how the `cross-entropy (CE) method` works in $2$ environments in Gym:
        - CartPole
        - the grid world of FrozenLake
    3. the theoretical background of the `cross-entropy (CE) method`

## 1. The taxonomy of RL methods

1. all the methods in RL can be classified into various groups:
    1. `model-free` or `model-based`
    2. `value-based` or `policy-based`
    3. `on-policy` or `off-policy`

2. there are other ways to taxonomize RL methods, but for now we are interested in the above $3$. Let's define them (as the specifics of our problem can influence our choice of a particular method):
    1. `model-free` or `model-based`:
        - `model-free`: the method doesn't build a model of the environment or reward; it just directly connects observations to actions (or values that are related to actions)
            - in other words, the agent takes current observations and does some computations on them, and the result is the action that it should take
            - usually easier to train because it's hard to build good models of complex environments with rich observations
        - `model-based`: try to predict what the next observation and/or reward will be
            - based on this prediction, the agent tries to choose the best possible action to take, very often making such predictions multiple times to look more and more steps into the future
            - pure `model-based` methods are usually used in deterministic environments (e.g. board games with strict rules)
        - all methods described in this book are from `model-free` category (as they have been the most active area of research for the past few year)
            - only recently have researchers started to mix the benefits from both worlds (refer to [Chapter20](../Chapter20/) and take a look of the `AlphaGo Zero` and `MuZero` methods, which use a model-based approach to board games and Atari)
    2. `value-based` or `policy-based`:
        - `policy-based` (PART 3): directly approximate the policy of the agent (what actions the agent should carry out at every step)
            - the policy is usually represented by a probability distribution over the available actions
        - `value-based` (PART 2): instead of the probability of actions, the agent calculates the value of every possible action and chooses the action with best value
    3. `on-policy` or `off-policy`: will be discussed in depth in PARTS 2 & 3
        - `off-policy`: is able to learn from historical data (obtained by a previous version of the agent, recorded by human demonstration, or just seen by the same agent several episodes ago)
        - `on-policy`: require fresh data for training, generated from the policy we're currently updating
            - they cannot be trained on old historical data because the result of the training will be wrong -> less data-efficient (we need much more communication with the environment)
            - but in some case, this is not a problem (the environment is very lightweight and fast, so we can quickly interact with it)

3. the `CE method` falls into the `model-free`, `policy-based`, and `on-policy` categories of methods, which means:
    1. it doesn't build a model of the environment; it just says to the agent what to do at every step
    2. it approximates the policy of the agent
    3. it requires fresh data obtained from the environment

## 2. The cross-entropy method in practice

1. Recall that the agent tries to accumulate as much total reward as possible by communicating with the environment:
    - in practice, we follow a common ML approach and replace all the complications of the agent with some kind of nonlinear function, which maps the agent's input (observations from the environment) to some output
    - output may depends on a particular method or a family of methods (such as `value-based` or `policy-based` methods)
    - as our `CE method` is `policy-based`, our nonlinear function (`neural network (NN)`) produces the `policy`, which basically says for every observation which action the agent should take
    - in research, `policy` is denoted as $\pi(a|s)$, where:
        - `a`: actions
        - `s`: the current states

2. A high-level approach to `policy-based` RL: ![A high-level approach to policy-based RL](../images/figure_4-1.png)
    - in practice, the policy is usually represented as a probability distribution over actions (very similar to a classification problem, with the number of classes being equal to the number of actions we can carry out)
    - this abstraction makes our agent very simple: it needs to pass an observation from the environment to the NN, get a probability distibution over actions, and perform random sampling using the probability distribution to get an action to carry out
        - this random sampling adds randomness to our agent (a good thing because at the begining of the training, when our weights are random, the agent behaves randomly)
        - as soon as the agent gets an action to issue, it fires the action to the environment and obtains the next observation and reward for the last action. Then the loop continues

3. during the agent's lifetime, its experience is presented as episodes:
    - every episode is a sequence of observations that the agent has got from the environment, actions it has issued, and rewards for these actions
        - imagine that our agent has played several episodes
        - for every episode, we can calculate the total reward that the agent has claimed (can be discounted or not discounted)
        - for simplicity, assumue a discount factor of $\gamma = 1$, i.e. an undiscounted sum of local rewards for every episode
    - this total reward shows how good this episode was for the agent
        - ![Example episodes with their observations, actions, and rewards](../images/figure_4-2.png) contains $4$ episodes (note that different episodes have different values for $o_i$, $a_i$, and $r_i$)
        - every cell represents the agent's step in the episode
        - due to randomness in the environment and the way that the agent selects actions to take, some episodes will be better thatn others
    - the core of the `CE method` is to throw away bad episode and train on better ones. The steps of `CE method`:
        1. play $N$ episodes using our current model and environment
        2. calculate the total reward for every episode and decide on a reward boundary
            - usually we use a percentile of all reward, e.g. $50{th}$ or $70^{th}$
        3. throw away all episodes with a reward below the boundary
        4. train on the remaining `"elite"` episodes (with rewards higher than the boundary) using observations as the input and issued actions as the desired output
        5. repear from step `1.` until we become satisfied with the result
    - with `CE method`, our NN learns how to repeat actions, which leads to a larger reward, constantly moving the boundary higher and higher

## 3. The cross-entropy method on CartPole

Refer to [cartpole script](01_cartpole.py):

In [6]:
#!/usr/bin/env python3
import numpy as np
import gymnasium as gym
from dataclasses import dataclass
import typing as tt
from torch.utils.tensorboard.writer import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class Net(nn.Module):
    def __init__(self, obs_size: int, hidden_size: int, n_actions: int):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x: torch.Tensor):
        return self.net(x)


@dataclass
class EpisodeStep:
    observation: np.ndarray
    action: int

@dataclass
class Episode:
    reward: float
    steps: tt.List[EpisodeStep]


def iterate_batches(env: gym.Env, net: Net, batch_size: int) -> tt.Generator[tt.List[Episode], None, None]:
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.tensor(obs, dtype=torch.float32)
        act_probs_v = sm(net(obs_v.unsqueeze(0)))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, is_trunc, _ = env.step(action)
        episode_reward += float(reward)
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done or is_trunc:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch: tt.List[Episode], percentile: float) -> \
        tt.Tuple[torch.FloatTensor, torch.LongTensor, float, float]:
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = float(np.percentile(rewards, percentile))
    reward_mean = float(np.mean(rewards))

    train_obs: tt.List[np.ndarray] = []
    train_act: tt.List[int] = []
    for episode in batch:
        if episode.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, episode.steps))
        train_act.extend(map(lambda step: step.action, episode.steps))

    train_obs_v = torch.FloatTensor(np.vstack(train_obs))
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


if __name__ == "__main__":
    # env = gym.make("CartPole-v1")
    env = gym.make("CartPole-v1", render_mode="rgb_array")
    env = gym.wrappers.RecordVideo(env, video_folder="video")
    assert env.observation_space.shape is not None
    obs_size = env.observation_space.shape[0]
    assert isinstance(env.action_space, gym.spaces.Discrete)
    n_actions = int(env.action_space.n)

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    print(net)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")

    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 475:
            print("Solved!")
            break
    writer.close()

Net(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)
0: loss=0.688, reward_mean=27.7, rw_bound=32.0
1: loss=0.679, reward_mean=26.1, rw_bound=31.0
2: loss=0.669, reward_mean=33.9, rw_bound=37.5
3: loss=0.647, reward_mean=46.6, rw_bound=52.0
4: loss=0.633, reward_mean=39.8, rw_bound=47.0
5: loss=0.625, reward_mean=39.5, rw_bound=52.0
6: loss=0.621, reward_mean=42.8, rw_bound=57.0
7: loss=0.623, reward_mean=53.6, rw_bound=57.5
8: loss=0.590, reward_mean=56.2, rw_bound=62.5
9: loss=0.592, reward_mean=54.3, rw_bound=64.0
10: loss=0.591, reward_mean=63.2, rw_bound=65.5
11: loss=0.593, reward_mean=77.4, rw_bound=94.0
12: loss=0.590, reward_mean=68.1, rw_bound=71.0
13: loss=0.582, reward_mean=62.9, rw_bound=73.0
14: loss=0.561, reward_mean=63.6, rw_bound=66.5
15: loss=0.560, reward_mean=57.6, rw_bound=63.5
16: loss=0.551, reward_mean=57.9, rw_bound=57.0
17: loss=0.566, reward_mea

1. **Line 13-15**: define constants/hyperparams:
    - **Line 13**: our model's core is a one-hidden-layer NN, with ReLU and $128$ hidden neurons (randomly set, not tuned)
    - **Line 14**: the count of episodes we play on every iteration ($16$)
    - **Line 15**: the percentile of each episode's total reward that we use for elite episode filtering; we'll keep the top $30%$ of episodes sorted by reward

2. **Line 18-28**: our NN takes a single observation from the environment and outputs a number for every action we can perform:
    - output is the probability distribution over actions
        - so a straightforward way to proceed would be to include softmax after the last layer
    - however, in the code, we don't apply softmax to increase the numerical stability of the training process. Rather than:
        1. calculating softmax (which uses exponention) and then
        2. calculating cross-entropy loss (which uses a logarithm of probas)
    - we'll use the Pytorch class `nn.CrossEntropyLoss` later, which combines both into a single, more numerically stable expression
        - `nn.CrossEntropyLoss` requires raw, unnormalized values from NN (called `logits`)
        - downside: we need to remember to apply softmax every time we need to get probas from NN's output

3. **Line 31-39**: define $2$ helper `dataclass`es. The purposes:
    1. `EpisodeStep`: used to represent one single step that our agent made in the episode, and it stores the observation from the environment and what action the agent performed
        - we'll use episode steps from elite episodes as training data
    2. `Episode`: a single episode stored as a total undiscounted reward and a collection of `EpisodeStep`

4. **Line 42-66**: `iterate_batch()` function generates batches with episodes:
    - **Line 42-47**: accept the environment (`Env` class instance), our NN, and the count of episodes it should generate on evey iteration
        - `batch`: used to accumulate our batch (which is a list of `Episode` instances)
        - `episode_reward`: a reward counter for the current episode
        - `episode_steps`: (the `EpisodeStep` object) list of steps
        - `obs`: first observation after `reset()`
        - `sm`: softmax layer, used to convert NN's output to a proba distribution of actions
    - **Line 48-51**: start the environment loop. At every iteration, we convert our current observation to a Pytorch tensor and pass it to NN to obtain action probas. Note:
        1. **Line 49-50**: all `nn.Module` instances expect a batch of data items, so we
            1. convert our observation into tensor and 
            2. add an extra dimension at the $0^{th}$ dim using `unsqueezed(0)` method, then 
            3. feed the raw action scores (aka NN's output) through softmax
        2. **Line 51**: both NN and softmax layer return tensors that track gradients, so we need to unpack this by accessing `tensor.data` field and converting to Numpy array, then we get the first batch element to obtain a one-dim vector of action probas
    - **Line 52-53**: use proba distribution of actions to obtain the actual action for the current step:
        1. **Line 52**: sample distribution using Numpy function `random.choice()`
        2. **Line 53**: pass this `action` to the environment to get our:
            1. next observation `next_obs`
            2. `reward`
            3. the indication of the episode ending `is_done` 
            4. the truncation flag `is_trunc`
            5. last value returned by `step()` function is extra info from the environment and is discarded
    - **Line 54-56**: `reward` is added to the current episode's total reward `episode_reward`, and our list of `episode_steps` is also extended with an `(observation, action)` pair
        - note: we save the obsevation that was used to choose the action (`obs`), but not the observation returned by the environment as a result of the action (`next_obs`)
    - **Line 57-66**: handles the situation when the current episode is over:
        1. in CartPole case, the episode ends when 
            1. the stick has fallen down
            2. time limit of the environment has been reached
        2. **Line 58-59**: we append the finalized episode to the batch, saving the total reward (as the episode has ben completed and we've accumulated all the rewards) and steps we have taken
        3. **Line 60-61**: then reset total reward accumulator `episode_reward` and clean the list of steps `episode_steps`
        4. **Line 62**: after that, we reset our environment and start over
        5. **Line 63-64**: if our batch has reached the desired count of episodes `batch_size`, we return it to the caller for processing using `yield`
            - our function is a generator, so every tume the `yield` operator is executed, the control is transferred to the outer iteration loop and then continues after the `yield` line
        6. **Line 65**: after processing, we'll clean up the `batch`
        7. **Line 66**: (IMPORTANT) assign an observation obtained from the environment `next_obs` to our current observation variable `obs`
        8. after that, everything repeats infinitely
            1. pass the observation to NN
            2. sample the action to perform
            3. ask the environment to process the action
            4. remember the result of this processing
        9. `ONE IMPORTANT FACT TO UNDERSTAND THIS FUNCTION'S LOGIT`: the training of NN and the generation of our episode are performed `at the same time`
            - they are not completely parallel, but every time our loop accumulates enough episodes (`16`), it passes control to this function caller, which is supposed to train the NN using gradient descent `GD`
            - so, when `yield` is returned, NN will have different, slightly better (we hope) behavior
            - as we should remember from the begining, `CE method` is from the `on-policy` class, so using fresh training data is important for the method to work properly
        10. since traing and data gathering in the same thread, proper synchronization isn't necessary
            - however, we should be aware of the frequent jumps between training the NN and using it
        
5. **Line 69-85**: define `filter_batch()` function htat lies at the core of the `CE method`:
    - **Line 69-73**: from the given `batch` of episodes and `percentile` value, it 
        1. **Line 71-72**: calculate a boundary reward `reward_bound` (used to filter elite episodes to train on) using Numpy's `percentile()` function
        2. **Line 73**: calculate `reward_mean` (used for monitoring)
    - **Line 75-81**: we'll filter off our episodes
        - for every episode in the batch, we'll check that the episode has a higher total reward than our boundary
        - if it has, we'll populate lists of observations and actions we'll train on
    - **Line 83-85**: convert our observations and actions from elite episodes into tensors, and return a tuple of $4$: observations, actions, the boundary of reward, and the mean reward
        - the last $2$ not used in traing; we'll write them in TensorBoard to check the performance of our agent

6. **Line 88-118**: let's glue everything together, and mostly consists of the training loop:
    - **Line 88-101**: create all the required objects:
        1. the environment
        2. NN
        3. the `objective()` function
        4. the optimizer
        5. the summary writer for TensorBoard
    - **Line 103-109**: in the training loop, we iterate our batches (a list of `Episode` objects):
        1. **Line 104**: `filter_batch()` filter the elite episodes, the result is
            1. tensors of observations
            2. taken actions
            3. the reward boundary used for filtering
            4. the mean reward
        2. **Line 105**: zero gradients of our NN
        3. **Line 106**: pass observations to the NN, obtaining its action scores
        4. **Line 107**: the `objective()` function uses these scores to calclulate the `CE` (between the NN output and the actions that the agent took)
            - the idea if this is to reinfoce our NN to carry out those elite actions that have led to good rewards
        5. **Line 108-109**: calculate gradients on the loss and ask the optimizer to adjust our NN
    - **Line 110-114**: monitor the progress
        1. **Line 110-111**: on the console, we show
            1. the iteration number
            2. the loss
            3. the mean reward of the batch
            4. the reward boundary
        2. **Line 112-114**: write the same values to TensorBoard to get a nice chart of the agent's learning performance
    - **Line 115-118**: compare the mean rewards of our batch episodes
        - when mean reward $> 475$, we stop our training. Why $475$?
            - in Gym, `CartPole-v1` environment is considered to be solved when mean reward for the last $100$ episodes is $>475$
            - however, our method converges so quickly that $100$ episodes are usually what we need
            - the properly trained agent can balance the stick for an infinitely long period of time (obtaining any amount of score), but the length of an episode in `CartPole-v1` is [limited to 500 steps](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/__init__.py)

7. **Line 90-91**: uncomment these lines to monitor the training process. We can tweak the environment by
    1. **Line 90**: setting the rendering mode in the CartPole environment, and
    2. **Line 91**: adding a `RecordVideo` wrapper to create a `/video` folder with a bunch of MP4 movies inside

8. REFLECTION:
    - our NN has learned how to play the environment purely from observations and rewards, without any interpretation of observed values
    - our implementation doesn't depend on environment-related details

9. start `TensorBoard` and look at the result by:
    1. open terminal
    2. type `tensorboard --logdir runs` inside `/Chapter04` folder
    3. open `http://localhost:6006/`
    4. run on a remote server and make it accessible from other machines: `tensorboard --logdir runs -bind_all`

10. Let's take a look of some visualizations:
    1. Mean reward: ![Cartpole mean reward](../images/ch04_cartpole_reward_mean.png)
    2. Loss: ![Cartpole loss](../images/ch04_cartpole_loss.png)
    3. Reward boundary during the training: ![Cartpole reward boundary during the training](../images/ch04_cartpole_reward_bound.png)
    4. Cartpole video episode $729$:

In [12]:
from IPython.display import Video

VIDEO_PATH = "video/rl-video-episode-729.mp4"

# With autoplay and loop
Video(VIDEO_PATH, 
      embed=True, # for self-contained notebook
      html_attributes="controls autoplay loop")

## 4. The cross-entropy method on FrozenLake

1. FrozenLake's world is of grid world category
    - agent lives in a grid of size $4\times 4$ and can move in $4$ directions
    - agent always starts at top-left and its goal is to reach bottom-right cell of the grid
    - there are holes in the fixed cells of the grid and if the agent gets into those holes, the episode ends and the reward is zero
    - if the agent reaches the bottom-right cell, the episode ends and the reward is $1.0$

2. the world is slipery, so the agent's actions do not always turn out as expected:
    - the agent will move in
        1. intended direction with probability specified by `success_rate` $(r)$
        2. either perpendicular direction with equal probability $\left(\frac{1-r}{2}\right)$ in both direction

3. Let's take a look of the environment: ![The FrozenLake environment rendered in human mode](../images/figure_4-6.png)

In [13]:
e =gym.make("FrozenLake-v1", render_mode="ansi")

print(f"Observation space: {e.observation_space}")
print(f"Action space: {e.action_space}")

Observation space: Discrete(16)
Action space: Discrete(4)


In [14]:
e.reset()

(0, {'prob': 1})

In [16]:
print(e.render())


SFFF
FHFH
FFFH
HFFG



Refer to [frozen lake naive script](02_frozenlake_naive.py):

In [17]:
#!/usr/bin/env python3
import numpy as np
import gymnasium as gym
from dataclasses import dataclass
import typing as tt
from torch.utils.tensorboard.writer import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res


class Net(nn.Module):
    def __init__(self, obs_size: int, hidden_size: int,
                 n_actions: int):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


@dataclass
class EpisodeStep:
    observation: np.ndarray
    action: int

@dataclass
class Episode:
    reward: float
    steps: tt.List[EpisodeStep]


def iterate_batches(env: gym.Env, net: Net, batch_size: int) -> \
        tt.Generator[tt.List[Episode], None, None]:
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.tensor(obs, dtype=torch.float32)
        act_probs_v = sm(net(obs_v.unsqueeze(0)))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, is_trunc, _ = env.step(action)
        episode_reward += float(reward)
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done or is_trunc:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch: tt.List[Episode], percentile: float) -> \
        tt.Tuple[torch.FloatTensor, torch.LongTensor, float, float]:
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = float(np.percentile(rewards, percentile))
    reward_mean = float(np.mean(rewards))

    train_obs: tt.List[np.ndarray] = []
    train_act: tt.List[int] = []
    for episode in batch:
        if episode.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation,
                             episode.steps))
        train_act.extend(map(lambda step: step.action,
                             episode.steps))

    train_obs_v = torch.FloatTensor(np.vstack(train_obs))
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


if __name__ == "__main__":
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    obs_size = env.observation_space.shape[0]
    n_actions = int(env.action_space.n)

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-frozenlake-naive")

    for iter_no, batch in enumerate(
            iterate_batches(env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = \
            filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 0.8:
            print("Solved!")
            break
    writer.close()

0: loss=1.377, reward_mean=0.1, reward_bound=0.0
1: loss=1.353, reward_mean=0.0, reward_bound=0.0
2: loss=1.329, reward_mean=0.0, reward_bound=0.0
3: loss=1.348, reward_mean=0.0, reward_bound=0.0
4: loss=1.331, reward_mean=0.0, reward_bound=0.0
5: loss=1.284, reward_mean=0.1, reward_bound=0.0
6: loss=1.230, reward_mean=0.0, reward_bound=0.0
7: loss=1.244, reward_mean=0.0, reward_bound=0.0
8: loss=1.295, reward_mean=0.0, reward_bound=0.0
9: loss=1.254, reward_mean=0.1, reward_bound=0.0
10: loss=1.218, reward_mean=0.1, reward_bound=0.0
11: loss=1.257, reward_mean=0.1, reward_bound=0.0
12: loss=1.219, reward_mean=0.0, reward_bound=0.0
13: loss=1.153, reward_mean=0.0, reward_bound=0.0
14: loss=1.162, reward_mean=0.0, reward_bound=0.0
15: loss=1.142, reward_mean=0.0, reward_bound=0.0
16: loss=1.170, reward_mean=0.0, reward_bound=0.0
17: loss=1.198, reward_mean=0.0, reward_bound=0.0
18: loss=1.205, reward_mean=0.1, reward_bound=0.0
19: loss=1.145, reward_mean=0.0, reward_bound=0.0
20: loss=1

KeyboardInterrupt: 

1. **Line 18-28**: implement `DiscreteOneHotWrapper` class/transformation as an (inherited from) `gym.ObservationWrapper`:
    - to minimize the required changes in our implemenetation, we can apply one-hot encoding of discrete inputs (i.e. the input to our NN will have $16$ float numbers and zero everywhere, except the index that we will encode that representing our current position on the grid)
    - with this wrapper applied to the environment, both the observation space and action space are $100%$ compatible with our CartPole solution
        - however we can see that our training doesn't improve the score over time

2. Let's take a look of some visualizations:
    1. Mean reward: ![FrozenLake naive mean reward](../images/ch04_frozenlake_naive_reward_mean.png)
    2. Loss: ![FrozenLake naive loss](../images/ch04_frozenlake_naive_loss.png)
    3. Reward boundary during the training: ![FrozenLake naive reward boundary during the training](../images/ch04_frozenlake_naive_reward_bound.png)

3. Let's take a deeper look at the reward structure of both environments:
    1. In Cartpole, every step of the environment gives us the reward $1.0$, until the moment that the pole falls
        - the longer our agent balanced the pole, the more reward it obtained
        - due to randomness in agent's behavior, different episodes were of different lengths, which gave us a pretty normal distribution of the episodes' rewards
        - after choosing a reward boundary, we rejected less successful episodes and learned how to repeat the better ones (by training on successful episodes' data)
        - reward distribution in the CartPole environment: ![Reward distribution in the CartPole environment](../images/figure_4-9.png)
    2. In FrozenLake, we get the reward of $1.0$ only when we reach the goal, and this reward says nothing about how good each episode was
        - the distribution of rewards for our episodes is also problematic
            - $2$ kinds of episodes possible, and failed episodes will obviously dominate at the beginning of the training, when the agent acts randomly
        - so our percentile selection of elite episodes is totally wrong and gives us bad examples to train on
        - reward distribution in the FrozenLake environment: ![Reward distribution in the FrozenLake environment](../images/figure_4-10.png)

4. This example shows us the limitations of the `CE method`:
    1. for training, our episodes have to be finite (in general, could be infinite) and preferably short
    2. total reward of the episodes should have enough variability to separate good episodes from bad ones
    3. it is beneficial to have an intermediate reward during the episode instead of having the reward at the end of the episode

5. A list of tweaks:
    1. `larger batches of played episodes`: require at leasr $100$ episodes just to get some successful episodes
    2. `discount factor applied to the reward`: use a discoundted total reward with the discount factor of $\gamma=0.9, 0.95$ to make the total reward for an episode depend on its length, and to add variety in episodes
        - reward for short episodes will be higher than for long ones
        - this increases variability in reward distribution
    3. `keeping elite episodes for a longer time`: a successful episode is rare --> keep for several iterations to train on them
    4. `decreasing the learning rate`: give NN time to average more training samples, as a smaller lr decreases the effect of new data on the model
    5. `much longer training time`: due to the sparsity of successful episodes and the random outcome of our actions, it's much harder for NN to get an idea of the best behavior to perform in any particular situation
        - to reach $50%$ successful episodes, about $5000$ training iterations are required

6. Refer to [fronzenlake tweaked script](03_frozenlake_tweaked.py):

In [18]:
#!/usr/bin/env python3
import numpy as np
import gymnasium as gym
from dataclasses import dataclass
import typing as tt
import random
from torch.utils.tensorboard.writer import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9



class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res


class Net(nn.Module):
    def __init__(self, obs_size: int, hidden_size: int,
                 n_actions: int):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


@dataclass
class EpisodeStep:
    observation: np.ndarray
    action: int

@dataclass
class Episode:
    reward: float
    steps: tt.List[EpisodeStep]


def iterate_batches(env: gym.Env, net: Net, batch_size: int) -> \
        tt.Generator[tt.List[Episode], None, None]:
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.tensor(obs, dtype=torch.float32)
        act_probs_v = sm(net(obs_v.unsqueeze(0)))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, is_trunc, _ = env.step(action)
        episode_reward += float(reward)
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done or is_trunc:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch: tt.List[Episode], percentile: float) -> \
        tt.Tuple[tt.List[Episode], tt.List[np.ndarray], tt.List[int], float]:
    reward_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(reward_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs: tt.List[np.ndarray] = []
    train_act: tt.List[int] = []
    elite_batch: tt.List[Episode] = []

    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation, example.steps))
            train_act.extend(map(lambda step: step.action, example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound


if __name__ == "__main__":
    random.seed(12345)
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)
    writer = SummaryWriter(comment="-frozenlake-tweaked")

    full_batch = []
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))
        full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
        if not full_batch:
            continue
        obs_v = torch.FloatTensor(np.vstack(obs))
        acts_v = torch.LongTensor(acts)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, rw_mean=%.3f, "
              "rw_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean,
            reward_bound, len(full_batch)))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)
        if reward_mean > 0.8:
            print("Solved!")
            break
    writer.close()

1: loss=1.380, rw_mean=0.020, rw_bound=0.000, batch=2
2: loss=1.370, rw_mean=0.000, rw_bound=0.000, batch=2
3: loss=1.362, rw_mean=0.000, rw_bound=0.000, batch=2
4: loss=1.378, rw_mean=0.040, rw_bound=0.000, batch=6
5: loss=1.375, rw_mean=0.000, rw_bound=0.000, batch=6
6: loss=1.379, rw_mean=0.040, rw_bound=0.000, batch=10
7: loss=1.377, rw_mean=0.000, rw_bound=0.000, batch=10
8: loss=1.378, rw_mean=0.020, rw_bound=0.000, batch=12
9: loss=1.376, rw_mean=0.010, rw_bound=0.000, batch=13
10: loss=1.374, rw_mean=0.010, rw_bound=0.000, batch=14
11: loss=1.373, rw_mean=0.000, rw_bound=0.000, batch=14
12: loss=1.374, rw_mean=0.020, rw_bound=0.000, batch=16
13: loss=1.375, rw_mean=0.020, rw_bound=0.000, batch=18
14: loss=1.373, rw_mean=0.010, rw_bound=0.000, batch=19
15: loss=1.374, rw_mean=0.010, rw_bound=0.000, batch=20
16: loss=1.376, rw_mean=0.040, rw_bound=0.000, batch=24
17: loss=1.376, rw_mean=0.000, rw_bound=0.000, batch=24
18: loss=1.377, rw_mean=0.020, rw_bound=0.000, batch=26
19: lo

KeyboardInterrupt: 

1. **Line 89-105**: change `filter_batch()` function to calculate the discounted reward and return elite episodes for us to keep

2. **Line 119-127**: in the training loop, store previous elite episodes to pass them to `filter_batch()` function on the next training iteration

3. the rest of the code is the same, except:
    1. `lr` decreases $10\times$
    2. `BATCHSIZE=100`

4. Let's take a look of some visualizations:
    1. Mean reward: ![FrozenLake tweaked mean reward](../images/ch04_frozenlake_tweaked_reward_mean.png)
    2. Loss: ![FrozenLake tweaked loss](../images/ch04_frozenlake_tweaked_loss.png)
    3. Reward boundary during the training: ![FrozenLake tweaked reward boundary during the training](../images/ch04_frozenlake_tweaked_reward_bound.png)

5. there are ways to address this (e.g. `entropy loss regularization`) (discuss later)

6. For nonslippery version, refer to [frozenlake nonslippery script](04_frozenlake_nonslippery.py):
    - **Line 112**: `is_slippery=False`

In [19]:
#!/usr/bin/env python3
import numpy as np
import gymnasium as gym
from dataclasses import dataclass
import typing as tt
import random
from torch.utils.tensorboard.writer import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9


class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res


class Net(nn.Module):
    def __init__(self, obs_size: int, hidden_size: int,
                 n_actions: int):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


@dataclass
class EpisodeStep:
    observation: np.ndarray
    action: int

@dataclass
class Episode:
    reward: float
    steps: tt.List[EpisodeStep]


def iterate_batches(env: gym.Env, net: Net, batch_size: int) -> \
        tt.Generator[tt.List[Episode], None, None]:
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.tensor(obs, dtype=torch.float32)
        act_probs_v = sm(net(obs_v.unsqueeze(0)))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, is_trunc, _ = env.step(action)
        episode_reward += float(reward)
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done or is_trunc:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch: tt.List[Episode], percentile: float) -> \
        tt.Tuple[tt.List[Episode], tt.List[np.ndarray],
                 tt.List[int], float]:
    reward_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(reward_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs: tt.List[np.ndarray] = []
    train_act: tt.List[int] = []
    elite_batch: tt.List[Episode] = []

    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation,
                                 example.steps))
            train_act.extend(map(lambda step: step.action,
                                 example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound


if __name__ == "__main__":
    random.seed(12345)
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1", is_slippery=False))
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)
    writer = SummaryWriter(comment="-frozenlake-nonslippery")

    full_batch = []
    for iter_no, batch in enumerate(iterate_batches(
            env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(
            lambda s: s.reward, batch))))
        full_batch, obs, acts, reward_bound = \
            filter_batch(full_batch + batch, PERCENTILE)
        if not full_batch:
            continue
        obs_v = torch.FloatTensor(np.vstack(obs))
        acts_v = torch.LongTensor(acts)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, rw_mean=%.3f, "
              "rw_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean,
            reward_bound, len(full_batch)))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)
        if reward_mean > 0.8:
            print("Solved!")
            break
    writer.close()

0: loss=1.410, rw_mean=0.020, rw_bound=0.000, batch=2
1: loss=1.403, rw_mean=0.000, rw_bound=0.000, batch=2
2: loss=1.397, rw_mean=0.000, rw_bound=0.000, batch=2
3: loss=1.388, rw_mean=0.030, rw_bound=0.000, batch=5
4: loss=1.382, rw_mean=0.000, rw_bound=0.000, batch=5
5: loss=1.378, rw_mean=0.020, rw_bound=0.000, batch=7
6: loss=1.373, rw_mean=0.000, rw_bound=0.000, batch=7
7: loss=1.362, rw_mean=0.020, rw_bound=0.000, batch=9
8: loss=1.353, rw_mean=0.040, rw_bound=0.000, batch=13
9: loss=1.348, rw_mean=0.010, rw_bound=0.000, batch=14
10: loss=1.342, rw_mean=0.000, rw_bound=0.000, batch=14
11: loss=1.335, rw_mean=0.040, rw_bound=0.000, batch=18
12: loss=1.330, rw_mean=0.000, rw_bound=0.000, batch=18
13: loss=1.335, rw_mean=0.040, rw_bound=0.000, batch=22
14: loss=1.335, rw_mean=0.040, rw_bound=0.000, batch=26
15: loss=1.333, rw_mean=0.040, rw_bound=0.000, batch=30
16: loss=1.329, rw_mean=0.030, rw_bound=0.000, batch=33
17: loss=1.327, rw_mean=0.040, rw_bound=0.000, batch=37
18: loss=1

1. Let's take a look of some visualizations:
    1. Mean reward: ![FrozenLake nonslippery mean reward](../images/ch04_frozenlake_nonslippery_reward_mean.png)
    2. Loss: ![FrozenLake nonslippery loss](../images/ch04_frozenlake_nonslippery_loss.png)
    3. Reward boundary during the training: ![FrozenLake nonslippery reward boundary during the training](../images/ch04_frozenlake_nonslippery_reward_bound.png)

## 5. The theoretical background of the cross-entropy method

- Refer to:
    1. book written by Reuven Rubinstein and Dirk P. Kroese `[RK04]`
    2. original paper by `Kroese`, title `Cross-entropy method, [Kro+11]`

1. The sampling theorem states:$$\begin{align*}
\mathbb{E}_{x\sim p(x)}[H(x)] &= \int_{x} p(x)H(x)dx = \int_{x} q(x)\frac{p(x)}{q(x)}H(x)dx \\
&= \mathbb{E}_{x\sim q(x)}\left[q(x)\frac{p(x)}{q(x)}H(x)\right]
\end{align*}$$ 
    - where (in RL case):
        - $H(x)$: a reward value obtained by some policy $x$
        - $p(x)$: a distribution of all possible policies
    - we don't want to maximize our reward by searching all possible policies; instead, we want to find a way to approximate $p(x)H(x)$ by $q(x)$, iterative minimizing the distance between them

2. The distance between $2$ probability distributions is calculated by `Kullback-Leibler (KL) divergence`:$$\begin{align*}
KL(p_1(x)\parallel p_2(x)) &= \mathbb{E}_{x\sim p_1(x)}\left[\log\frac{p_1(x)}{p_2(x)}\right] \\
&= \mathbb{E}_{x\sim p_1(x)}\left[\log p_1(x)\right] - \mathbb{E}_{x\sim p_1(x)}\left[\log p_2(x)\right]
\end{align*}$$ 
    - where:
        - $\mathbb{E}_{x\sim p_1(x)}\left[\log p_1(x)\right]$: called `entropy`, and it doesn't depend on $p_2(x)$, so it could be omitted during the minimization
        - $\mathbb{E}_{x\sim p_1(x)}\left[\log p_2(x)\right]$: called `cross-entropy`

3. Combining both formulas, we can get an iterative algorithm. This is an approximation of $p(x)H(x)$ with an update:$$\begin{align*}
q_0(x) &= p(x) \\
q_{i+1}(x) &= \argmin_{q_{i+1}(x)} \left( -\mathbb{E}_{x\sim q_{i}(x)}\left[\frac{p(x)}{q_{i}(x)}H(x)\log q_{i+1}(x)\right] \right)
\end{align*}$$
    - this is a generic cross-entropy method that can be significant simplified in our RL case

4. Replace $H(x)$ with an indicator function ($1$ when the reward for the episode is above the threshold, and $0$ when the reward is below). Our policy update:$$\begin{align*}
\pi_{i+1}(a|s) &= \argmin_{\pi_{i+1}} \left( -\mathbb{E}_{z\sim \pi_{i}(a|s)}\left[\left[R(z)\geq \psi_{i}\right]\log\pi_{i+1}(a|s)\right] \right)
\end{align*}$$
    - strictly speaking, the formula misses the normalization term, but it still works in practice without it
    - the method is quite clear: 
        1. we sample episodes using our current policy (starting with some random inital policy), and
        2. minimize the negative log likelihood of the most successful samples and our policy